## Load the PDF file

In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('../data/Evolution_of_AI.pdf')

## Create Chunks

In [2]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)
pages = loader.load_and_split(text_splitter)

## Fetch and use key from .env

In [3]:
import os
import google.generativeai as genai
from dotenv import load_dotenv

load_dotenv()
genai.configure(api_key = os.getenv('GEMINI_API'))
api_key = os.getenv('GEMINI_API')

c:\Users\devas\Documents\Projects\new\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create Embeddings

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)

## Store in Chroma DB

In [5]:
from langchain.vectorstores import Chroma

# Turn the chunks into embeddings and store them in Chroma
vectordb=Chroma.from_documents(pages, embeddings)
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

## Create LLM for Prompts

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key = api_key)

## Define prompt template

In [7]:
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

# Create the retrieval chain
template = """
You are a helpful AI assistant.
Answer based on the context provided. 
If you don't know, say I don't know
context: {context}
input: {input}
answer:
"""
prompt = PromptTemplate.from_template(template)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

## Write Prompts

In [8]:
response=retrieval_chain.invoke({"input":"What is personalised learning environments?"})

response["answer"]

'Constructivist principles also underscore the importance of personalised learning, recognising the diversity of learners and their unique cognitive processes (Dede, 2010). AI in education excels at creating personalised learning environments by adapting content and instructional strategies based on individual student needs (Russel & Norvig, 2010). Machine learning algorithms analyse student data to identify strengths, weaknesses, and learning preferences, tailoring educational materials accordingly. This individualisation of learning experiences fosters a more student-centric approach, where learners have the autonomy to explore topics at their own pace. Research in this domain underscores the positive impact of personalised learning environments on student motivation, engagement, and knowledge acquisition, reinforcing the constructivist tenet that learners actively shape their understanding of the material (Siemens, 2005).'

In [9]:
response=retrieval_chain.invoke({"input":"What is the use of AI in Education?"})

response["answer"]

'AI in education is used for enhancing the efficiency and effectiveness of educational processes, catering to individual learning needs. It encompasses a diverse range of applications, including intelligent tutoring systems, adaptive learning platforms, and virtual reality simulations.'

In [10]:
response=retrieval_chain.invoke({"input":"What is AI?"})

response["answer"]

"I don't know"